<a href="https://colab.research.google.com/github/xxxnmxhj00/Mini-Project/blob/main/%5BTF%5Dkor_bert_question_answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers
!pip install --upgrade tf_keras

In [ ]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

--2024-09-19 04:29:10--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M  --.-KB/s    in 0.1s    

2024-09-19 04:29:11 (330 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2024-09-19 04:29:11--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
import tensorflow as tf

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
import os
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf

In [ ]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions =[]
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

In [ ]:
train_contexts, train_questions, train_answers = read_squad('KorQuAD_v1.0_train.json')
val_contexts, val_questions, val_answers = read_squad('KorQuAD_v1.0_dev.json')

In [ ]:
print('훈련 데이터의 본문 개수 :', len(train_contexts))
print('훈련 데이터의 질문 개수 :', len(train_questions))
print('훈련 데이터의 답변 개수 :', len(train_answers))
print('테스트 데이터의 본문 개수 :', len(val_contexts))
print('테스트 데이터의 질문 개수 :', len(val_questions))
print('테스트 데이터의 답변 개수 :', len(val_answers))

훈련 데이터의 본문 개수 : 60407
훈련 데이터의 질문 개수 : 60407
훈련 데이터의 답변 개수 : 60407
테스트 데이터의 본문 개수 : 5774
테스트 데이터의 질문 개수 : 5774
테스트 데이터의 답변 개수 : 5774


In [ ]:
print('첫번째 샘플의 본문')
print('------------------')
print(train_contexts[0])

첫번째 샘플의 본문
------------------
1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.


In [ ]:
print('첫번째 샘플의 질문')
print('------------------')
print(train_questions[0])

첫번째 샘플의 질문
------------------
바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?


In [ ]:
print('첫번째 샘플의 답변')
print('------------------')
print(train_answers[0]) # 시작하는 위치와 끝나는 위치를 예측할거라했음

첫번째 샘플의 답변
------------------
{'text': '교향곡', 'answer_start': 54}


In [ ]:
def add_end_index(answers, contexts):
    for answer, context in zip(answers, contexts):
        answer['text'] = answer['text'].rstrip()
      # 좌우 공백 없애는 함수
        gold_text = answer['text'] # 입력받은 답변
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        assert context[start_idx:end_idx] == gold_text
        answer['answer_end'] = end_idx
add_end_index(train_answers, train_contexts)
add_end_index(val_answers, val_contexts)
# 예측을 시작텍스트와 종료 텍스트로 예상하면 됨

In [ ]:
print('첫번째 샘플의 답변')
print('------------------')
print(train_answers[0])

첫번째 샘플의 답변
------------------
{'text': '교향곡', 'answer_start': 54, 'answer_end': 57}


In [ ]:
train_contexts[0][54]

'교'

In [ ]:
train_contexts[0][56]

'곡'

In [ ]:
train_contexts[0][54:57]

'교향곡'

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    deleting_list = []


    for i in tqdm (range(len(answers))):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
            deleting_list.append(i)

        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
            if i not in deleting_list:
                deleting_list.append(i)

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return deleting_list

In [ ]:
deleting_list_for_train = add_token_positions(train_encodings, train_answers)
deleting_list_for_test = add_token_positions(val_encodings, val_answers)

100%|██████████| 5774/5774 [00:00<00:00, 374496.06it/s]


In [ ]:
# 순서가 맞지 않는 데이터는 삭제
def delete_samples(encodings, deleting_list):
  input_ids = np.delete(np.array(encodings['input_ids']), deleting_list, axis = 0)
  attention_mask = np.delete(np.array(encodings['attention_mask']), deleting_list, axis = 0)
  start_positions = np.delete(np.array(encodings['start_positions']), deleting_list, axis = 0)
  end_positions = np.delete(np.array(encodings['end_positions']), deleting_list, axis = 0)

  X_data = [input_ids, attention_mask] # Fix the typo here
  y_data = [start_positions, end_positions]

  return X_data, y_data

In [ ]:
X_train, y_train = delete_samples(train_encodings, deleting_list_for_train)
X_test, y_test = delete_samples(val_encodings, deleting_list_for_test)

In [ ]:
from transformers import TFBertModel

In [ ]:
# 모델 층을 변환
class TFBertForQuestionAnswering(tf.keras.Model):
  def __init__(self, model_name):
      super(TFBertForQuestionAnswering, self).__init__()
      self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
      self.qa_outputs = tf.keras.layers.Dense(2,
                                              kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                              name='qa_outputs')
      self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)

  def call(self, inputs):
      input_ids, attention_mask = inputs
      outputs = self.bert(input_ids, attention_mask=attention_mask)

      sequence_output = outputs[0]

      logits = self.qa_outputs(sequence_output)
      start_logits, end_logits = tf.split(logits, 2, axis=-1)

      start_logits = tf.squeeze(start_logits, axis=-1)
      end_logits = tf.squeeze(end_logits, axis=-1)

      start_probs = self.softmax(start_logits)
      end_probs = self.softmax(end_logits)

      return start_probs, end_probs

In [ ]:

model = TFBertForQuestionAnswering('klue/bert-base')
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer=optimizer, loss=loss)



pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'bert.embeddings.position_ids', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=1,
    batch_size=4
)

15035/15035 [==============================] - 7640s 505ms/step - loss: 1.3500 - output_1_loss: 0.6138 - output_2_loss: 0.7362


In [ ]:
# 저장
model.save_pretrained('qna_model/bert-base')


AttributeError: 'TFBertForQuestionAnswering' object has no attribute 'save_pretrained'